In [1]:
import numpy as np
import json
from StringIO import StringIO
import pandas as pd
from __future__ import division
import matplotlib.pyplot as plt
import urllib2,base64
import cPickle as pickle
from collections import Counter
from collections import defaultdict
from collections import OrderedDict
from __future__ import division
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import geoplotlib as gp
import scipy as sp
import datetime
import re
import os
import os.path
import requests
from geoplotlib.utils import BoundingBox
from sklearn import neighbors, datasets,linear_model,preprocessing
from sklearn.preprocessing import scale
from pylab import *
from scipy import stats
from sklearn.neighbors import DistanceMetric
from urllib2 import urlopen, Request
from pprint import pprint
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from collections import defaultdict

from pushover import init, Client

%matplotlib inline

### Trim Rough Data Option

In [2]:
#Kept only US weather stations

with open('ghcnd-stations.txt') as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content if x[0:2]=="US"]

f = open("ghcnd-us-stations.txt", "w")
f.write("\n".join(content))
f.close()



In [3]:
temp_stations = pd.read_csv('weather_station_list.csv',skipinitialspace=True)
temp_stations['GHCND']=temp_stations.GHCND.str.strip()
temp_stations=temp_stations[temp_stations.GHCND.str.contains("US")] ##Discard non US stations
temp_stations_clean=temp_stations[np.abs(temp_stations.LON-temp_stations.LON.mean())<=(5*temp_stations.LON.std())] ##discard OUTLIER by location

print "include outliner, size:",temp_stations.shape
print "exclude outliner, size:",temp_stations_clean.shape

include outliner, size: (56918, 6)
exclude outliner, size: (56911, 6)


In [15]:
#Use folium to plot interactive map of stations
if a=b:
USA_COORDINATES = (40, -95.7129)
popups, locations = [], []

width, height = 210,50

usa_station_map = folium.Map(location=USA_COORDINATES, zoom_start=3,
                   tiles='Stamen Terrain')


for index, row in temp_stations_clean.iterrows():
    locations.append([row['LAT'], row['LON']])
    #popups.append(iframe)

t = folium.FeatureGroup(name='WeatherStation')
t.add_child(MarkerCluster(locations=locations))
usa_station_map.add_child(t)


usa_station_map.save('usa_station_map.html')
usa_station_map

KeyboardInterrupt: 

In [4]:
##LOAD ALL YEARLY DATA INTO A DICTONARY (LONG EXECUTION TIM!!!!!!)

file_17=os.getcwd()+'\\data\\data option two\\2017.csv'
file_16=os.getcwd()+'\\data\\data option two\\2016.csv'
file_15=os.getcwd()+'\\data\\data option two\\2015.csv'
file_14=os.getcwd()+'\\data\\data option two\\2014.csv'
file_13=os.getcwd()+'\\data\\data option two\\2013.csv'
file_12=os.getcwd()+'\\data\\data option two\\2012.csv'
file_11=os.getcwd()+'\\data\\data option two\\2011.csv'
file_10=os.getcwd()+'\\data\\data option two\\2010.csv'
file_09=os.getcwd()+'\\data\\data option two\\2009.csv'
file_08=os.getcwd()+'\\data\\data option two\\2008.csv'
file_lib=[file_17,file_16,file_15,file_14,file_13,file_12,file_11,file_10,file_09,file_08]
file_data_dict=dict()
used_columns = [0,1,2,3,7]
data_columns = ['GHCND', 'DATE', 'ATTRIBUTE', 'VALUE', 'TIME']
dtypes = ['object']+['object']+['object']+[float, 'object']
for yearly_file, year in zip(file_lib,range(2017, 2007, -1)):
    yearly_data = pd.read_csv(yearly_file,header=None,usecols = used_columns,
                              dtype=dict(zip(used_columns, dtypes)),sep=',').rename(columns=dict(zip(used_columns, data_columns)))
    file_data_dict[year]=yearly_data

In [5]:
#data analysis based on 20170601 - 20170801  (LONG EXECUTION TIM!!!!!!)

yearly_data=file_data_dict[2017]
date_rng=[d.strftime('%Y%m%d') for d in pd.date_range('2017-6-1', periods=61, freq='D')] #range from 6-1 to 7-30
daily_station_list=list()
for date in date_rng:
    daily_station=yearly_data[yearly_data['GHCND'].isin(temp_stations_clean['GHCND'].tolist())
                               & (yearly_data['DATE']==date) & (yearly_data['ATTRIBUTE'].isin(["TMAX"])) ]
    #print "# of station reported in ",date," ",len(daily_station['GHCND'].unique().tolist())
    daily_station_list.append(daily_station['GHCND'].unique().tolist())

intersected_stations = set(daily_station_list[0]).intersection(*daily_station_list)
print "# of station reported from 20170601 to 20170731 ",len(intersected_stations)

# of station reported from 20170601 to 20170731  4776


In [57]:
reported_meta_stations=temp_stations_clean[temp_stations_clean['GHCND'].isin(intersected_stations)] 
for date in date_rng:
    reported_data_stations=yearly_data[yearly_data['GHCND'].isin(list(intersected_stations)) & (yearly_data['DATE']==date) 
                                    & (yearly_data['ATTRIBUTE'].isin(["TMAX"]))]
    reported_full_stations=pd.merge(reported_meta_stations, reported_data_stations, on='GHCND', how='outer')
    print reported_full_stations.shape
    reported_full_stations.head(3200).to_csv('data\\daily dataframe\\'+date+'stations.csv', index=False)


(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)
(4776, 10)


In [30]:
##Read daily summary data for this year 2017
file_name=os.getcwd()+'\\data\\data option two\\2017.csv'
used_columns = [0,1,2,3,7]
data_columns = ['GHCND', 'DATE', 'ATTRIBUTE', 'VALUE', 'TIME']
dtypes = ['object']+[int]+['object']+[float, 'object']
yearly_data = pd.read_csv(file_name,header=None,usecols = used_columns,
                   dtype=dict(zip(used_columns, dtypes)),sep=',').rename(columns=dict(zip(used_columns, data_columns)))
yearly_data.head(10)

,GHCND,DATE,ATTRIBUTE,VALUE,TIME
0,ASN00015643,20170101,TMAX,274.0,NaN
1,ASN00015643,20170101,TMIN,218.0,NaN
2,ASN00015643,20170101,PRCP,2.0,NaN
3,CA1MB000296,20170101,PRCP,0.0,NaN
4,US1NCBC0113,20170101,PRCP,5.0,NaN
5,US1MTMH0019,20170101,PRCP,43.0,NaN
6,US1MTMH0019,20170101,SNOW,28.0,NaN
7,US1MTMH0019,20170101,SNWD,178.0,NaN
8,ASN00085296,20170101,TMAX,217.0,NaN
9,ASN00085296,20170101,TMIN,127.0,NaN


In [26]:
#data analysis based on 20170701 - 20170801

date=20170701
#print temp_stations_clean['GHCND'].tolist()
lastmonth_station=yearly_data[yearly_data['GHCND'].isin(temp_stations_clean['GHCND'].tolist())
                               & (yearly_data['DATE']==date) & (yearly_data['ATTRIBUTE'].isin(["TAVG","TMIN","TMAX"]))]
print lastmonth_station.shape
print lastmonth_station.head()


print "# of station reported in July 2017 07 01 ",len(lastmonth_station['GHCND'].unique().tolist())

(15591, 5)
                GHCND      DATE ATTRIBUTE  VALUE TIME
16512259  USW00094626  20170701      TMAX  189.0  NaN
16512260  USW00094626  20170701      TMIN  161.0  NaN
16512271  USW00014719  20170701      TMAX  244.0  NaN
16512272  USW00014719  20170701      TMIN  206.0  NaN
16512280  USW00024061  20170701      TMAX  294.0  NaN
# of station reported in July 2017 07 01  6650


In [34]:
lastmonth_st_list=lastmonth_station['GHCND'].unique().tolist()
temp_stations_lastmonth=temp_stations_clean[temp_stations_clean['GHCND'].isin(lastmonth_st_list) & (temp_stations_clean['ST'].str.strip()!='AK')] ##except alaska
temp_stations_lastmonth.to_csv('20170701stations.csv', index=False)

In [15]:
#Use Geoplotlib to plot static map of stations
latitute_list = temp_stations['LAT']
longitude_list = temp_stations['LON']
geo_data_for_plotting = {"lat": latitute_list,
                         "lon": longitude_list}

In [97]:
max_lat, min_lat = 49.5904, 24.9493
max_lon, min_lon =  -66.9326,-125.0011 
bbox = BoundingBox(north=max_lat, west=min_lon, south=min_lat, east=max_lon)
gp.set_bbox(bbox)

gp.dot(geo_data_for_plotting, point_size=0.5)
gp.inline()